In [96]:
import re
import pandas as pd

In [7]:
from tika import parser

# Đọc file doc
file_path = "(VN) Chú giải HS 2022.doc"
parsed = parser.from_file(file_path)

# In nội dung từng dòng
content = parsed["content"]
content_list = content.splitlines()


2025-01-02 13:56:08,133 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [8]:
content_list = [x.strip() for x in content_list if x.strip()]

In [9]:
content_list

['PHỤ LỤC',
 'PHỤ LỤC',
 'CHÚ GIẢI CHI TIẾT DANH MỤC HS2022 (EN2022)',
 '(Ban hành kèm theo công văn số 1810/TCHQ-TXNK ngày 26 tháng 4 năm 2024 của Tổng cục Hải quan)',
 'PHẦN I:',
 'ĐỘNG VẬT SỐNG; CÁC SẢN PHẨM TỪ ĐỘNG VẬT',
 'Chú giải.',
 '1. Trong phần này, khi đề cập đến một giống hoặc một loài động vật, trừ khi có yêu cầu khác, cần phải kể đến cả giống hoặc loài động vật đó còn non.',
 '2. Trừ khi có yêu cầu khác, trong toàn bộ Danh mục này bất cứ đề cập nào liên quan đến các sản phẩm "được làm khô" cũng bao gồm các sản phẩm được khử nước, làm bay hơi hoặc làm khô bằng đông lạnh.',
 'Chương 1:',
 'Động vật sống',
 'Chú giải.',
 '1. Chương này bao gồm tất cả các loại động vật sống trừ:',
 '(a) Cá và động vật giáp xác, động vật thân mềm và động vật thuỷ sinh không xương sống khác, thuộc nhóm 03.01, 03.06, 03.07 hoặc 03.08;',
 '(b) Vi sinh vật nuôi cấy và các sản phẩm khác thuộc nhóm 30.02; và',
 '(c) Động vật thuộc nhóm 95.08.',
 'TỔNG QUÁT',
 'Chương này bao gồm tất cả các loại sinh

In [50]:
section_re = r"^[Pp][Hh][Ầ ầ][Nn]\s+[IVX]+:?$"
chapter_re = r"^chương\s([1-9]|[1-9][0-9]):?$"
heading_re = "\d{2}\.\d{2}\s\-\s"

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\nguye\AppData\Local\Temp\ipykernel_5832\1742796502.py:3: SyntaxWarning: invalid escape sequence '\d'
  heading_re = "\d{2}\.\d{2}\s\-\s"


In [63]:
section_start_index = [index for index,text in enumerate(content_list) if re.search(section_re, text)]

In [117]:
chapter_start_index = [index for index,text in enumerate(content_list) if re.search(chapter_re, text.lower())]

In [108]:
heading_start_index = [index for index,text in enumerate(content_list) if re.search(heading_re, text.lower())]

In [109]:
chugiai_re = r"^Chú\sgiải\.?$"


In [110]:
section_indexs = []
for section_start_idx in section_start_index:
    # Lọc các phần tử trong chapter_start_index lớn hơn section_start_idx
    larger_ = [section_end_idx for section_end_idx in chapter_start_index if section_end_idx > section_start_idx]
    if larger_:  
        section_indexs.append([section_start_idx, min(larger_)])

In [94]:
all_section_info = []

for section in section_indexs:
    section_info= {}
    section_content = content_list[section[0]:section[1]]
    # print(section)
    
    section_info['section_name'] =  content_list[section[0]]
    section_info['vi_description'] = content_list[section[0]+1:section[1]]
    section_info['vi_notes'] = []
    for index, text in enumerate(section_content):
        match = re.search(chugiai_re, text)
        if match:
            section_info['section_name'] =  content_list[section[0]]
            section_info['vi_description'] = content_list[section[0]+1:section[0]+index]
            section_info['vi_notes'] = content_list[section[0]+index :section[1]]
    # print(section_info)
    all_section_info.append(section_info)

In [98]:
# Chuyển dữ liệu thành DataFrame
df = pd.DataFrame(all_section_info)

# Ghi DataFrame ra file Excel
df.to_excel('vi_sections_notes_2024.xlsx', index=False)

In [177]:
phanchuong1_re = r"^phân\schương\s+[1iI]$"

In [178]:
phanchuong1_index = [index for index,text in enumerate(content_list) if re.search(phuongchuong1_re, text.lower())]

In [179]:
phanchuong1_index

[5253, 7249, 16634, 24499, 25401, 26175, 27074]

In [176]:
thêm logic 
    Nếu giữa 2 chapter_start_idx có phanchuong1_index thì lấy đến phanchuong1_index, nếu không có

SyntaxError: invalid syntax (619668860.py, line 1)

In [ ]:
chapter_index = []
for chapter_start_idx in chapter_start_index:
    # Lọc các phần tử trong heading_start_index lớn hơn chapter_start_idx
    larger_ = [chapter_end_idx for chapter_end_idx in heading_start_index if chapter_end_idx > chapter_start_idx]
    if larger_:
        chapter_index.append([chapter_start_idx, min(larger_)])

In [147]:
all_chapter_info = []
for chapter in chapter_index:
    chapter_info= {}
    chapter_info['chapter_name'] =  content_list[chapter[0]]
    chapter_info['vi_description'] = content_list[chapter[0]+1]
    chapter_info['vi_notes'] = content_list[chapter[0]+2:chapter[1]]
    chapter_info['vi_general'] = []
    
    chapter_content = content_list[chapter[0]:chapter[1]]
    for idx,content in enumerate(chapter_content):
        # print(idx,content)
        if content == 'TỔNG QUÁT':
            # print(idx)
            chapter_info['vi_notes'] = content_list[chapter[0]+2:chapter[0]+idx]
            chapter_info['vi_general'] = content_list[chapter[0]+idx:chapter[1]]


    all_chapter_info.append(chapter_info)

In [148]:
all_chapter_info

[{'chapter_name': 'Chương 1:',
  'vi_description': 'Động vật sống',
  'vi_notes': ['Chú giải.',
   '1. Chương này bao gồm tất cả các loại động vật sống trừ:',
   '(a) Cá và động vật giáp xác, động vật thân mềm và động vật thuỷ sinh không xương sống khác, thuộc nhóm 03.01, 03.06, 03.07 hoặc 03.08;',
   '(b) Vi sinh vật nuôi cấy và các sản phẩm khác thuộc nhóm 30.02; và',
   '(c) Động vật thuộc nhóm 95.08.'],
  'vi_general': ['TỔNG QUÁT',
   'Chương này bao gồm tất cả các loại sinh vật sống (dùng làm thực phẩm hoặc có các mục đích khác) trừ:',
   '(1) Cá và động vật giáp xác, động vật thân mềm và động vật thủy sinh không xương sống khác.',
   '(2) Vi sinh vật nuôi cấy và các sản phẩm khác thuộc nhóm 30.02.',
   '(3) Bầy thú thuộc rạp xiếc, bầy thú hoặc động vật làm trò lưu động tương tự khác (nhóm 95.08).',
   'Động vật, kể cả côn trùng, chết trên đường vận chuyển được phân loại vào các nhóm từ 02.01 đến 02.05, 02.07, 02.08 hoặc 04.10 nếu chúng là những động vật ăn được thích hợp dùng l